In [13]:
import kagglehub

path = kagglehub.dataset_download("dev0914sharma/dataset")

print("Path to dataset files:", path)

Path to dataset files: /home/shahwaiz-mughal/.cache/kagglehub/datasets/dev0914sharma/dataset/versions/1


In [14]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [15]:
df = pd.read_csv("Dataset.csv")

df.head()

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742


In [16]:
df.isnull().sum()

user_id      0
item_id      0
rating       0
timestamp    0
dtype: int64

In [17]:
df.drop(columns=['timestamp'], inplace=True)

In [18]:
scaler = MinMaxScaler()
df['rating'] = scaler.fit_transform(df[['rating']])

df.head()

,user_id,item_id,rating
0,0,50,1.0
1,0,172,1.0
2,0,133,0.0
3,196,242,0.5
4,186,302,0.5


In [19]:
user_item_matrix = df.pivot(index="user_id", columns="item_id", values="rating").fillna(0)

user_item_matrix.head()


item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.00,0.5,0.75,0.5,0.5,1.0,0.75,0.0,1.0,0.50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.75,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
user_similarity = cosine_similarity(user_item_matrix)

user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

user_similarity_df.head()


user_id,0,1,2,3,4,5,6,7,8,9,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
0,1.000000,0.118314,0.124035,0.000000,0.167248,0.161615,0.051571,0.078272,0.240990,0.177471,...,0.149487,0.000000,0.061186,0.168133,0.105336,0.000000,0.154406,0.000000,0.189832,0.120434
1,0.118314,1.000000,0.170598,0.038133,0.056890,0.363306,0.425077,0.427316,0.327894,0.080052,...,0.360361,0.105531,0.276900,0.181519,0.186162,0.112330,0.305234,0.150539,0.171256,0.380570
2,0.124035,0.170598,1.000000,0.085467,0.171143,0.078752,0.229212,0.104713,0.102751,0.156840,...,0.156445,0.306546,0.351634,0.419692,0.316834,0.231843,0.225032,0.147493,0.169235,0.108301
3,0.000000,0.038133,0.085467,1.000000,0.312274,0.024633,0.059608,0.052691,0.064280,0.059172,...,0.029905,0.039652,0.145070,0.044846,0.107704,0.018466,0.134995,0.088810,0.124476,0.026770
4,0.167248,0.056890,0.171143,0.312274,1.000000,0.030891,0.058939,0.079480,0.178854,0.103886,...,0.053128,0.037294,0.127916,0.182779,0.145343,0.029773,0.197985,0.147171,0.164698,0.055392


In [21]:
def recommend_movies(user_id, user_similarity_df, user_item_matrix, top_n=5):
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:]  

    weighted_ratings = user_item_matrix.loc[similar_users.index].T.dot(similar_users)

    similarity_sums = similar_users.sum()
    weighted_ratings = weighted_ratings / similarity_sums

    already_rated = user_item_matrix.loc[user_id]
    recommendations = weighted_ratings[already_rated == 0].sort_values(ascending=False)

    return recommendations.head(top_n)  


In [22]:
InputUserId = 1


In [23]:
recommended_movies = recommend_movies(InputUserId, user_similarity_df, user_item_matrix)
recommended_movies

item_id
318    0.396549
423    0.346375
357    0.345681
483    0.329765
286    0.318832
dtype: float64